In [2]:
import pymc3 as pm
import theano
import theano.tensor as tt

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('ggplot')

In [13]:
df = pd.read_csv('weekly_dmd.csv')
df['date'] = pd.to_datetime(df.date).apply(lambda x: x.date())
df = df.set_index(df.date)
df.index.rename('date_idx', inplace=True)
df.head(2)

,zone_id,date,total_demand_new,total_demand_new_lag1,total_demand_new_lag2,total_demand_new_lag3,total_demand_new_lag-1,total_demand_new_lag-2,total_demand_new_lag-3,smoothed_total_demand,year
date_idx,,,,,,,,,,,
2017-06-26,1,2017-06-26,1656.912757,NaN,NaN,NaN,1678.654277,2193.532475,2110.494687,NaN,2017
2017-06-26,655,2017-06-26,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,2017


In [14]:
covid_start_idx = date(2020,2,17)

df_prep = df.loc[(df.date < covid_start_idx)
                | (df.date >= covid_start_idx + timedelta(weeks=52))].sort_values('date')

df_prep['y_obs'] = df_prep.total_demand_new
# df_prep['y_obs'] = df_prep.smoothed_total_demand
df_prep['covid_start_m1'] = df_prep.date.apply(lambda x: 1.0 if x == covid_start_idx - timedelta(weeks=1) else 0)

In [ ]:
# [] remove first X (3?) weeks for each zone?
# [] how to align ish
    # should be aligned temporally
    # can access the season estimates based on time t then?
    # will have to do a theano.scan for each of the zones -- for zones that started after the oldest zone, do they not get initial seasons then? are their initial seasons just the 52 prior 

# maybe initial level and initial trend can be unique to each zone, but other params come from a shared distribution

# so the "parent" distribution for initial szns should be ordered/accessed based on time

# so s'pose we have
# szn_len = 4
#  t: 0,1,2,3,4,5,...
# z1: 3,4,8,2,5,6,...
# z2: _,_,_,1,3,4,...

# initial_szns = s1,s2,s3,s4

# the first non-zero entry for z2 is the 4th season
# so the initial_szns passed to z2's theano.scan should be ordered as [s4,s1,s2,s3]

# s1_mu ~ Uniform[0,1]
# s1_sigma ~ HalfCauchy(0.5)
# s1_mu_z1 ~ TruncatedNormal(s1_mu, s1_sigma, upper=1, lower=0)


In [ ]:
zone_list = dd.zone_id.unique()
dd['zone_idx'] = dd.zone_id.apply(lambda x: np.where(zone_list == x)[0][0])

# does this need to be mapped to [0,len(dd.zone_id.unique())-1] ? seems yes
zone_idx = dd.zone_idx.values

In [ ]:
df_prep

In [105]:
def fit_hw_model(zdf, initial_lvl_upper):
    szn_len = 52

    with pm.Model() as hw_model:
    #     initial_level = pm.Normal(f'initial_level', mu=3000, sigma=400)
#         initial_level = pm.Uniform(f'initial_level', lower=0, upper=5000)
        initial_level = pm.Uniform(f'initial_level', lower=0, upper=initial_lvl_upper, shape=(df_prep.zone_id.nunique()))
    
        smoothing_level_mu = pm.Uniform('smoothing_level_mu', lower=0, upper=1)
        smoothing_level_sigma = pm.HalfCauchy('smoothing_level_sigma', beta=0.5)
        smoothing_level = pm.TruncatedNormal('smoothing_level', mu=smoothing_level_mu, sigma=smoothing_level_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_level = pm.Normal('smoothing_level', mu=0.5, sigma=0.5)

    #     initial_trend = pm.Normal(f'initial_trend', mu=0, sigma=100)
#         initial_trend = pm.Uniform(f'initial_trend', lower=-1000, upper=1000)
        initial_trend = pm.Uniform(f'initial_trend', lower=-initial_lvl_upper/2, upper=initial_lvl_upper/2)
        
        smoothing_trend_mu = pm.Uniform('smoothing_trend_mu', lower=0, upper=1)
        smoothing_trend_sigma = pm.HalfCauchy('smoothing_trend_sigma', beta=0.5)
        smoothing_trend = pm.TruncatedNormal('smoothing_trend', mu=smoothing_trend_mu, sigma=smoothing_trend_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_trend = pm.Uniform('smoothing_trend', lower=0, upper=1)
    #     smoothing_trend = pm.Normal('smoothing_trend', mu=0.5, sigma=0.5)
    
        
        # 
        initial_szns_mus = pm.Normal('initial_szns_mus', mu=1.0, sigma=0.2, shape=(szn_len))
        initial_szns = pm.Normal('initial_szns', mu=initial_szns_mus, sigma=0.2, shape=(df_prep.zone_id.nunique(), szn_len))
    #     initial_szns = pm.Uniform('initial_szns', lower=0.7, upper=1.3, shape=(szn_len))
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1-smoothing_level)
        smoothing_season_mu = pm.Uniform('smoothing_season_mu', lower=0, upper=1)
        smoothing_season_sigma = pm.HalfCauchy('smoothing_season_sigma', beta=0.5)
        smoothing_season = pm.TruncatedNormal('smoothing_season', mu=smoothing_season_mu, sigma=smoothing_season_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_season = pm.TruncatedNormal('smoothing_season', mu=0.2, sigma=0.5, lower=0, upper=0.8)
    #     smoothing_season = pm.Normal('smoothing_season', mu=0.5, sigma=0.5)

        ys = tt.as_tensor_variable(zdf['y_obs'])
        is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
    #     covid_level_change = pm.Normal('covid_level_change', mu=15000, sigma=2000)
        covid_level_change = pm.Uniform('covid_level_change', lower=0, upper=5000)
#         covid_level_change = pm.Normal('covid_level_change', mu=2000, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=20000, sigma=2000)
        covid_level_change2 = pm.Uniform('covid_level_change2', lower=0, upper=5000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)

        def hw_component_wise(y, covid_start, prior_level, prior_trend, szn_m52):
            updated_szn = smoothing_season * y / (prior_level + prior_trend) + (1 - smoothing_season) * szn_m52


            next_level = smoothing_level * (y/szn_m52) + (1 - smoothing_level)*(prior_level + prior_trend) + covid_start*covid_level_change
            next_trend = smoothing_trend * (next_level - prior_level) + (1 - smoothing_trend)*prior_trend - covid_start*smoothing_trend*covid_level_change2


            return next_level, next_trend, updated_szn

        # pass initial_level[zone_idx], initial_trend[zone_idx], initial_szns[zone_idx] etc
        # should the scan be wrapped in a scan too??
            # the outer scan would loop through each zone and pass each zone's initial params and data to the inner scan that calls hw_component_wise
        outputs, updates = theano.scan(hw_component_wise,
               sequences = [
                   ys[1:],
#                    ys[szn_len:],
                   is_covid_start[1:]
#                    is_covid_start[szn_len:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = initial_level, taps=None),
                   dict(initial = initial_trend, taps=None),
                   dict(initial = initial_szns, taps=[-szn_len])
               ], 
        #            non_sequences = 
               )


        levels = outputs[0]
        trends = outputs[1]
        seasons = outputs[2]


        levels_f = pm.math.concatenate([initial_level.reshape(1,1), levels[:-1]])
        trends_f = pm.math.concatenate([initial_trend.reshape(1,1), trends[:-1]])
        seasons_f = pm.math.concatenate([initial_szns, seasons[:-szn_len]])

        levels_and_trends = pm.math.stack([levels_f, trends_f])
        level_plus_trend = levels_and_trends.sum(axis=0)

        level_plus_trend_and_seasons = pm.math.stack([level_plus_trend, seasons_f])
        y_hats = level_plus_trend_and_seasons.prod(axis=0)

        sig = pm.HalfCauchy('sigma', beta=10)
#         y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[szn_len-1:-1, :]['y_obs_lag-1'])
        y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[:-1, :]['y_obs_lag-1'])
        
    map_estimate = pm.find_MAP(model=hw_model)
    
    return map_estimate, ys, is_covid_start

In [15]:
d2 = df_prep.loc[df_prep.zone_id.isin([1,13])]

In [18]:
zone_list = d2.zone_id.unique()
d2['zone_idx'] = d2.zone_id.apply(lambda x: np.where(zone_list == x)[0][0])

# does this need to be mapped to [0,len(dd.zone_id.unique())-1] ? seems yes
zone_idx = d2.zone_idx.values

<ipython-input-18-8e0f7511f979>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2['zone_idx'] = d2.zone_id.apply(lambda x: np.where(zone_list == x)[0][0])


In [17]:
d2.head()

,zone_id,date,total_demand_new,total_demand_new_lag1,total_demand_new_lag2,total_demand_new_lag3,total_demand_new_lag-1,total_demand_new_lag-2,total_demand_new_lag-3,smoothed_total_demand,year,y_obs,covid_start_m1
date_idx,,,,,,,,,,,,,
2017-06-26,1,2017-06-26,1656.912757,NaN,NaN,NaN,1678.654277,2193.532475,2110.494687,NaN,2017,1656.912757,0.0
2017-06-26,13,2017-06-26,272.496914,NaN,NaN,NaN,277.941595,354.757040,345.466023,NaN,2017,272.496914,0.0
2017-07-03,13,2017-07-03,277.941595,272.496914,NaN,NaN,354.757040,345.466023,350.206164,NaN,2017,277.941595,0.0
2017-07-03,1,2017-07-03,1678.654277,1656.912757,NaN,NaN,2193.532475,2110.494687,2194.632691,NaN,2017,1678.654277,0.0
2017-07-10,13,2017-07-10,354.757040,277.941595,272.496914,NaN,345.466023,350.206164,360.568384,311.527674,2017,354.757040,0.0


In [212]:
# let's test that we can do the hierarchical scanning business
szn_len = 4
d = pd.DataFrame({'zone_id': [1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2],
#                   'y_obs': [1,2,3,4,2,3,4,5,3,1,2,3,4,1],
                  'y_obs': [1,2,3,4,2,3,4,5,3,0,0,0,0,1,2,3,4,1],
                  't': [1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]})

In [213]:
zone_list = d.zone_id.unique()
d['zone_idx'] = d.zone_id.apply(lambda x: np.where(zone_list == x)[0][0])
d['covid_start_m1'] = d.t.apply(lambda x: 1 if x == 7 else 0)

# does this need to be mapped to [0,len(dd.zone_id.unique())-1] ? seems yes
zone_idx = d.zone_idx.values

In [214]:
d

,zone_id,y_obs,t,zone_idx,covid_start_m1
0,1,1,1,0,0
1,1,2,2,0,0
2,1,3,3,0,0
3,1,4,4,0,0
4,1,2,5,0,0
5,1,3,6,0,0
6,1,4,7,0,1
7,1,5,8,0,0
8,1,3,9,0,0
9,2,0,1,1,0


In [12]:
np.unique(zone_idx)

array([0, 1])

In [ ]:
# so the "parent" distribution for initial szns should be ordered/accessed based on time

# so s'pose we have
# szn_len = 4
#  t: 0,1,2,3,4,5,...
# z1: 3,4,8,2,5,6,...
# z2: _,_,_,1,3,4,...

# initial_szns = s1,s2,s3,s4

# the first non-zero entry for z2 is the 4th season
# so the initial_szns passed to z2's theano.scan should be ordered as [s4,s1,s2,s3]

In [ ]:
# w/ what ive been doing, we havent worried about which szn corresponds to which t, we just start from wherever there is demand (or 3rd week w/ demand)
# now though, we should always index on a particular t/week #
# ie make initial_szns[:, 0] correspond to first week in year, initial_szns[:, 51] corresponding to last, etc
# so given the first_t, we want:

initial_szns[z_idx, first_t%52:szn_len] + initial_szns[z_idx, :first_t%52]

In [21]:
[1,2,3,4][2:4]

[3, 4]

In [22]:
[1,2,3,4][:2]

[1, 2]

In [ ]:
# [0,...,51]
# []

In [20]:
52%52, 53%52

(0, 1)

In [90]:
theano.config.compute_test_value = 'raise'

In [97]:
# def test(x, prev_output):
def test(x):
    return pm.math.floatX(x**2)

outputs, updates = theano.scan(test,
                               sequences=[
                                   np.array([1.0,2.0,3.0,4.0])
#                                    tt.as_tensor_variable([1.0,2.0,3.0,4.0])
                               ],
#                               outputs_info=[
#                                   dict(initial=pm.math.floatX(0), taps=None)
#                               ]
                              )

In [98]:
outputs

for{cpu,scan_fn}.0

In [99]:
outputs.get_test_value()

array([ 1.,  4.,  9., 16.])

In [299]:
szn_len

4

In [314]:
with pm.Model() as test_hw_model:
#     initial_level = pm.Normal(f'initial_level', mu=3000, sigma=400)
#         initial_level = pm.Uniform(f'initial_level', lower=0, upper=5000)
    initial_lvl_upper = 10000
    initial_level = pm.Uniform(f'initial_level', lower=0, upper=initial_lvl_upper, shape=(d.zone_id.nunique()))

    smoothing_level_mu = pm.Uniform('smoothing_level_mu', lower=0, upper=1)
    smoothing_level_sigma = pm.HalfCauchy('smoothing_level_sigma', beta=0.5)
    smoothing_level = pm.TruncatedNormal('smoothing_level', mu=smoothing_level_mu, sigma=smoothing_level_sigma, lower=0, upper=1, shape=(d.zone_id.nunique()))
#         smoothing_level = pm.Normal('smoothing_level', mu=0.5, sigma=0.5)

#     initial_trend = pm.Normal(f'initial_trend', mu=0, sigma=100)
#         initial_trend = pm.Uniform(f'initial_trend', lower=-1000, upper=1000)
    initial_trend = pm.Uniform(f'initial_trend', lower=-initial_lvl_upper/2, upper=initial_lvl_upper/2, shape=(d.zone_id.nunique()))

    smoothing_trend_mu = pm.Uniform('smoothing_trend_mu', lower=0, upper=1)
    smoothing_trend_sigma = pm.HalfCauchy('smoothing_trend_sigma', beta=0.5)
    smoothing_trend = pm.TruncatedNormal('smoothing_trend', mu=smoothing_trend_mu, sigma=smoothing_trend_sigma, lower=0, upper=1, shape=(d.zone_id.nunique()))
#         smoothing_trend = pm.Uniform('smoothing_trend', lower=0, upper=1)
#     smoothing_trend = pm.Normal('smoothing_trend', mu=0.5, sigma=0.5)


    # 
    initial_szns_mus = pm.Normal('initial_szns_mus', mu=1.0, sigma=0.2, shape=(szn_len))
    initial_szns = pm.Normal('initial_szns', mu=initial_szns_mus, sigma=0.2, shape=(d.zone_id.nunique(), szn_len))
#     initial_szns = pm.Uniform('initial_szns', lower=0.7, upper=1.3, shape=(szn_len))
#     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
#     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
#     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1-smoothing_level)
    smoothing_season_mu = pm.Uniform('smoothing_season_mu', lower=0, upper=1)
    smoothing_season_sigma = pm.HalfCauchy('smoothing_season_sigma', beta=0.5)
    smoothing_season = pm.TruncatedNormal('smoothing_season', mu=smoothing_season_mu, sigma=smoothing_season_sigma, lower=0, upper=1, shape=(d.zone_id.nunique()))
#         smoothing_season = pm.TruncatedNormal('smoothing_season', mu=0.2, sigma=0.5, lower=0, upper=0.8)
#     smoothing_season = pm.Normal('smoothing_season', mu=0.5, sigma=0.5)

#     ys = tt.as_tensor_variable(zdf['y_obs'])
#     is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])

#     covid_level_change = pm.Normal('covid_level_change', mu=15000, sigma=2000)
    covid_level_change = pm.Uniform('covid_level_change', lower=0, upper=5000, shape=(d.zone_id.nunique()))
#         covid_level_change = pm.Normal('covid_level_change', mu=2000, sigma=1000)
#     covid_level_change2 = pm.Normal('covid_level_change2', mu=20000, sigma=2000)
    covid_level_change2 = pm.Uniform('covid_level_change2', lower=0, upper=5000, shape=(d.zone_id.nunique()))

In [216]:
initial_szns.get_test_value()

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [217]:
## prep data
ys_list, is_covid_start_list, first_t_list =  [], [], []

for z_idx in np.unique(zone_idx):
    zdf = d.loc[d.zone_idx == z_idx]
    
    ys_list.append(zdf['y_obs'].tolist())
    is_covid_start_list.append(zdf['covid_start_m1'].tolist())
    first_t_list.append(zdf.t.min())

In [221]:
ys_list

[[1, 2, 3, 4, 2, 3, 4, 5, 3], [0, 0, 0, 0, 1, 2, 3, 4, 1]]

In [222]:
is_covid_start_list

[[0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0]]

In [223]:
first_t_list

[1, 1]

In [162]:
initial_level.get_test_value()

array([5000., 5000.])

In [164]:
initial_level.get_test_value().shape

(2,)

In [163]:
initial_level.get_test_value()[zone_idx]

array([5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000.,
       5000., 5000., 5000., 5000., 5000.])

In [161]:
initial_level[zone_idx].get_test_value()

array([5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000.,
       5000., 5000., 5000., 5000., 5000.])

In [205]:
ys_list

[[1, 2, 3, 4, 2, 3, 4, 5, 3], [1, 2, 3, 4, 1]]

In [170]:
tt.as_tensor_variable(np.array([1,2,3]))

TensorConstant{[1 2 3]}

In [192]:
np.unique(zone_idx)

array([0, 1])

In [208]:
tt.as_tensor_variable([[1, 2, 3, 4, 2, 3, 4, 5, 3], [0,0,0,0,1, 2, 3, 4, 1]])

TensorConstant{[[1 2 3 4 .. 2 3 4 1]]}

In [211]:
tt.as_tensor_variable([[1, 2, 3, 4, 2, 3, 4, 5, 3], [0,0,0,0,1, 2, 3, 4, 1]])[1].get_test_value()

array([0, 0, 0, 0, 1, 2, 3, 4, 1])

In [204]:
tt.as_tensor_variable([[1,2],[3,4]])

TensorConstant{[[1 2]
 [3 4]]}

In [199]:
tt.as_tensor_variable(ys_list[0])

TensorConstant{[1 2 3 4 2 3 4 5 3]}

In [225]:
np.array(ys_list)

array([[1, 2, 3, 4, 2, 3, 4, 5, 3],
       [0, 0, 0, 0, 1, 2, 3, 4, 1]])

In [249]:
# def test(x, prev_output):
def test(x,y,o1,o2,o3):
#     return pm.math.floatX(x**2), pm.math.floatX(3.0)
    return pm.math.floatX(x**2), pm.math.floatX(x**2), pm.math.floatX(x**2)

# outputs, updates = theano.scan(test,
#                                sequences=[
#                                    np.array([1.0,2.0,3.0,4.0])
# #                                    tt.as_tensor_variable([1.0,2.0,3.0,4.0])
#                                ],
# #                               outputs_info=[
# #                                   dict(initial=pm.math.floatX(0), taps=None)
# #                               ]
#                               )

In [292]:
smoothing_level.get_test_value()

array([0.5, 0.5])

In [315]:
def hw_component_wise_wrapper(z_idx):
    def hw_component_wise(y, covid_start, prior_level, prior_trend, szn_m52):
        updated_szn = smoothing_season[z_idx] * y / (prior_level + prior_trend) + (1 - smoothing_season[z_idx]) * szn_m52


        next_level = smoothing_level[z_idx] * (y/szn_m52) + (1 - smoothing_level[z_idx])*(prior_level + prior_trend) + covid_start*covid_level_change[z_idx]
        next_trend = smoothing_trend[z_idx] * (next_level - prior_level) + (1 - smoothing_trend[z_idx])*prior_trend - covid_start*smoothing_trend[z_idx]*covid_level_change2[z_idx]


        return next_level, next_trend, updated_szn
    return hw_component_wise

In [316]:
covid_level_change[0].get_test_value()

array(2500.)

In [311]:
smoothing_season[0].get_test_value()

array(0.5)

In [310]:
smoothing_level[0].get_test_value()*(1/3) + (1-smoothing_level[0].get_test_value())*(1+2)

1.6666666666666667

In [300]:
fn = hw_component_wise_wrapper(0)

In [303]:
x=fn(1,0,1,2,3)
x

(Elemwise{add,no_inplace}.0,
 Elemwise{sub,no_inplace}.0,
 Elemwise{add,no_inplace}.0)

In [304]:
x[0].get_test_value()

array([1.66666667, 1.66666667, 1.66666667, 1.66666667])

In [305]:
x[1].get_test_value()

array([1.33333333, 1.33333333, 1.33333333, 1.33333333])

In [306]:
x[2].get_test_value()

array(1.66666667)

In [297]:
initial_szns[1].get_test_value()

array([1., 1., 1., 1.])

In [317]:
# def inner_scan_for_zone(zone_idx, ys, is_covid_start, first_t):
def inner_scan_for_zone(z_idx, ys, is_covid_start):
#     zdf = d.loc[d.zone_idx == zone_idx]
#     ys = tt.as_tensor_variable(zdf['y_obs'])
#     is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
#     first_t = zdf.t.min()
#     initial_seasons = initial_szns[zone_idx, first_t%szn_len:szn_len] + initial_szns[zone_idx, :first_t%szn_len]

#     initial_seasons = pm.math.concatenate([initial_szns[zone_idx, first_t%szn_len:szn_len], initial_szns[zone_idx, :first_t%szn_len]])
#     import pdb; pdb.set_trace()

#     outputs, updates = theano.scan(test,
#                                     sequences=[
#                                         ys[1:],
#                                         is_covid_start[1:]
#                                     ],
#                                    outputs_info=[
#                                        dict(initial = initial_level[z_idx], taps=None),
#                                        dict(initial = initial_trend[z_idx], taps=None),
#                                        dict(initial = initial_szns[z_idx], taps=[-szn_len])
#                                    ])
    
#     outputs, updates = theano.scan(hw_component_wise,
    outputs, updates = theano.scan(hw_component_wise_wrapper(z_idx),
                                  sequences=[
                                      ys[1:], # this [1:] is done assuming we padded and shifted y_obs
                                      is_covid_start[1:]
                                  ],
                                  outputs_info=[
                                      dict(initial = initial_level[z_idx], taps=None),
                                      dict(initial = initial_trend[z_idx], taps=None),
                                      dict(initial = initial_szns[z_idx], taps=[-szn_len])
#                                       dict(initial = initial_seasons, taps=[-szn_len])
                                  ])
    return outputs

outputs, updates = theano.scan(inner_scan_for_zone,
           sequences=[
               tt.as_tensor_variable(np.unique(zone_idx)),
#                np.unique(zone_idx),
               tt.as_tensor_variable(ys_list),
#                np.array(ys_list),
               tt.as_tensor_variable(is_covid_start_list),
#                np.array(is_covid_start_list),
#                np.array(first_t_list)
           ])

WARNING (theano.scan.basic): Output None (index 0) has a initial state but taps is explicitly set to None 
WARNING (theano.scan.basic): Output None (index 1) has a initial state but taps is explicitly set to None 


In [ ]:
# apply the padding for ys[1:], etc
# confirm ok that we're padding newer zones with 0's (initial level will be 0 i guess, but should be fine -- since the optimal smoothing_level will still be determined by the non-0 data)

In [318]:
outputs

[for{cpu,scan_fn}.0, for{cpu,scan_fn}.1, for{cpu,scan_fn}.2]

In [319]:
outputs[0].get_test_value()

array([[2501.        ,  627.25      , -465.1875    , -895.109375  ,
        -883.79416827, 1845.30301136,  876.55212782,  173.53125729],
       [2500.        ,  625.        , -468.75      , -897.9375    ,
        -886.296875  , 1843.09765625,  874.52050781,  170.60236347]])

In [287]:
outputs[2].get_test_value()

array([[ 4.,  9., 16.,  4.,  9., 16., 25.,  9.],
       [ 0.,  0.,  0.,  1.,  4.,  9., 16.,  1.]])

In [265]:
tt.as_tensor_variable(np.unique(zone_idx)).get_test_value()

array([0, 1])

In [269]:
tt.as_tensor_variable(np.unique(zone_idx)).dimshuffle(1,'x').get_test_value()

ValueError: new_order[0] is 1, but the input only has 1 axes.

In [268]:
x=tt.as_tensor_variable(np.unique(zone_idx))

In [270]:
x.padleft

AttributeError: 'TensorConstant' object has no attribute 'padleft'

In [257]:
outputs[0].get_test_value()

array([[ 4.,  9., 16.,  4.,  9., 16., 25.,  9.],
       [ 0.,  0.,  0.,  1.,  4.,  9., 16.,  1.]])

In [111]:
for z_idx in np.unique(zone_idx):
    zdf = d2.loc[d2.zone_idx == zone_idx]
    ys = tt.as_tensor_variable(zdf['y_obs'])
    is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
    first_t = zdf.t.min()
    initial_seasons = initial_szns[zone_idx, first_t%szn_len:szn_len] + initial_szns[zone_idx, :first_t%szn_len]
    
    outputs, updates = theano.scan(hw_component_wise,
               sequences = [
                   ys[1:],
#                    ys[szn_len:],
                   is_covid_start[1:]
#                    is_covid_start[szn_len:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = initial_level[zone_idx], taps=None),
                   dict(initial = initial_trend[zone_idx], taps=None),
                   dict(initial = initial_seasons, taps=[-szn_len])
#                    dict(initial = initial_szns[z_idx], taps=[-szn_len])
               ], 
        #            non_sequences = 
               )

    return outputs

SyntaxError: 'return' outside function (<ipython-input-111-1840a6545aa7>, line 25)

In [ ]:
def fit_hw_model(zdf, initial_lvl_upper):
    szn_len = 52

    with pm.Model() as hw_model:
    #     initial_level = pm.Normal(f'initial_level', mu=3000, sigma=400)
#         initial_level = pm.Uniform(f'initial_level', lower=0, upper=5000)
        initial_level = pm.Uniform(f'initial_level', lower=0, upper=initial_lvl_upper, shape=(df_prep.zone_id.nunique()))
    
        smoothing_level_mu = pm.Uniform('smoothing_level_mu', lower=0, upper=1)
        smoothing_level_sigma = pm.HalfCauchy('smoothing_level_sigma', beta=0.5)
        smoothing_level = pm.TruncatedNormal('smoothing_level', mu=smoothing_level_mu, sigma=smoothing_level_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_level = pm.Normal('smoothing_level', mu=0.5, sigma=0.5)

    #     initial_trend = pm.Normal(f'initial_trend', mu=0, sigma=100)
#         initial_trend = pm.Uniform(f'initial_trend', lower=-1000, upper=1000)
        initial_trend = pm.Uniform(f'initial_trend', lower=-initial_lvl_upper/2, upper=initial_lvl_upper/2)
        
        smoothing_trend_mu = pm.Uniform('smoothing_trend_mu', lower=0, upper=1)
        smoothing_trend_sigma = pm.HalfCauchy('smoothing_trend_sigma', beta=0.5)
        smoothing_trend = pm.TruncatedNormal('smoothing_trend', mu=smoothing_trend_mu, sigma=smoothing_trend_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_trend = pm.Uniform('smoothing_trend', lower=0, upper=1)
    #     smoothing_trend = pm.Normal('smoothing_trend', mu=0.5, sigma=0.5)
    
        
        # 
        initial_szns_mus = pm.Normal('initial_szns_mus', mu=1.0, sigma=0.2, shape=(szn_len))
        initial_szns = pm.Normal('initial_szns', mu=initial_szns_mus, sigma=0.2, shape=(df_prep.zone_id.nunique(), szn_len))
    #     initial_szns = pm.Uniform('initial_szns', lower=0.7, upper=1.3, shape=(szn_len))
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1-smoothing_level)
        smoothing_season_mu = pm.Uniform('smoothing_season_mu', lower=0, upper=1)
        smoothing_season_sigma = pm.HalfCauchy('smoothing_season_sigma', beta=0.5)
        smoothing_season = pm.TruncatedNormal('smoothing_season', mu=smoothing_season_mu, sigma=smoothing_season_sigma, lower=0, upper=1, shape=(df_prep.zone_id.nunique()))
#         smoothing_season = pm.TruncatedNormal('smoothing_season', mu=0.2, sigma=0.5, lower=0, upper=0.8)
    #     smoothing_season = pm.Normal('smoothing_season', mu=0.5, sigma=0.5)

        ys = tt.as_tensor_variable(zdf['y_obs'])
        is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
    #     covid_level_change = pm.Normal('covid_level_change', mu=15000, sigma=2000)
        covid_level_change = pm.Uniform('covid_level_change', lower=0, upper=5000)
#         covid_level_change = pm.Normal('covid_level_change', mu=2000, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=20000, sigma=2000)
        covid_level_change2 = pm.Uniform('covid_level_change2', lower=0, upper=5000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)

        def hw_component_wise(y, covid_start, prior_level, prior_trend, szn_m52):
            updated_szn = smoothing_season * y / (prior_level + prior_trend) + (1 - smoothing_season) * szn_m52


            next_level = smoothing_level * (y/szn_m52) + (1 - smoothing_level)*(prior_level + prior_trend) + covid_start*covid_level_change
            next_trend = smoothing_trend * (next_level - prior_level) + (1 - smoothing_trend)*prior_trend - covid_start*smoothing_trend*covid_level_change2


            return next_level, next_trend, updated_szn

        # pass initial_level[zone_idx], initial_trend[zone_idx], initial_szns[zone_idx] etc
        # should the scan be wrapped in a scan too??
            # the outer scan would loop through each zone and pass each zone's initial params and data to the inner scan that calls hw_component_wise
        outputs, updates = theano.scan(hw_component_wise,
               sequences = [
                   ys[1:],
#                    ys[szn_len:],
                   is_covid_start[1:]
#                    is_covid_start[szn_len:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = initial_level, taps=None),
                   dict(initial = initial_trend, taps=None),
                   dict(initial = initial_szns, taps=[-szn_len])
               ], 
        #            non_sequences = 
               )


        levels = outputs[0]
        trends = outputs[1]
        seasons = outputs[2]


        levels_f = pm.math.concatenate([initial_level.reshape(1,1), levels[:-1]])
        trends_f = pm.math.concatenate([initial_trend.reshape(1,1), trends[:-1]])
        seasons_f = pm.math.concatenate([initial_szns, seasons[:-szn_len]])

        levels_and_trends = pm.math.stack([levels_f, trends_f])
        level_plus_trend = levels_and_trends.sum(axis=0)

        level_plus_trend_and_seasons = pm.math.stack([level_plus_trend, seasons_f])
        y_hats = level_plus_trend_and_seasons.prod(axis=0)

        sig = pm.HalfCauchy('sigma', beta=10)
#         y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[szn_len-1:-1, :]['y_obs_lag-1'])
        y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[:-1, :]['y_obs_lag-1'])
        
    map_estimate = pm.find_MAP(model=hw_model)
    
    return map_estimate, ys, is_covid_start

In [ ]:
### dampened

# when training, should we train on the horizon of interest?

In [ ]:
def fit_hw_model(zdf, initial_lvl_upper):
    szn_len = 52

    with pm.Model() as hw_model:
    #     initial_level = pm.Normal(f'initial_level', mu=3000, sigma=400)
#         initial_level = pm.Uniform(f'initial_level', lower=0, upper=5000)
        initial_level = pm.Uniform(f'initial_level', lower=0, upper=initial_lvl_upper)
        smoothing_level = pm.Uniform('smoothing_level', lower=0, upper=1)
#         smoothing_level = pm.Normal('smoothing_level', mu=0.5, sigma=0.5)

    #     initial_trend = pm.Normal(f'initial_trend', mu=0, sigma=100)
#         initial_trend = pm.Uniform(f'initial_trend', lower=-1000, upper=1000)
        initial_trend = pm.Uniform(f'initial_trend', lower=-initial_lvl_upper/2, upper=initial_lvl_upper/2)
        smoothing_trend = pm.Uniform('smoothing_trend', lower=0, upper=1)
    #     smoothing_trend = pm.Normal('smoothing_trend', mu=0.5, sigma=0.5)

        initial_szns = pm.Normal('initial_szns', mu=1.0, sigma=0.2, shape=(szn_len))
    #     initial_szns = pm.Uniform('initial_szns', lower=0.7, upper=1.3, shape=(szn_len))
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1)
    #     smoothing_season = pm.Uniform('smoothing_season', lower=0, upper=1-smoothing_level)
        smoothing_season = pm.TruncatedNormal('smoothing_season', mu=0.2, sigma=0.5, lower=0, upper=0.8)
    #     smoothing_season = pm.Normal('smoothing_season', mu=0.5, sigma=0.5)
    
        dampener = pm.Uniform('dampener', lower=0, upper=1)

        ys = tt.as_tensor_variable(zdf['y_obs'])
        is_covid_start = tt.as_tensor_variable(zdf['covid_start_m1'])
    #     covid_level_change = pm.Normal('covid_level_change', mu=15000, sigma=2000)
        covid_level_change = pm.Uniform('covid_level_change', lower=0, upper=5000)
#         covid_level_change = pm.Normal('covid_level_change', mu=2000, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=20000, sigma=2000)
        covid_level_change2 = pm.Uniform('covid_level_change2', lower=0, upper=5000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)
    #     covid_level_change2 = pm.Normal('covid_level_change2', mu=covid_level_change, sigma=1000)

        def hw_component_wise(y, covid_start, prior_level, prior_trend, szn_m52):
            updated_szn = smoothing_season * y / (prior_level + dampener*prior_trend) + (1 - smoothing_season) * szn_m52


            next_level = smoothing_level * (y/szn_m52) + (1 - smoothing_level)*(prior_level + dampener*prior_trend) + covid_start*covid_level_change
            next_trend = smoothing_trend * (next_level - prior_level) + (1 - smoothing_trend)*dampener*prior_trend - covid_start*smoothing_trend*covid_level_change2


            return next_level, next_trend, updated_szn

        outputs, updates = theano.scan(hw_component_wise,
               sequences = [
                   ys[1:],
#                    ys[szn_len:],
                   is_covid_start[1:]
#                    is_covid_start[szn_len:]
        #            post_covid[szn_len:]
               ],
               outputs_info = [
                   dict(initial = initial_level, taps=None),
                   dict(initial = initial_trend, taps=None),
                   dict(initial = initial_szns, taps=[-szn_len])
               ], 
        #            non_sequences = 
               )


        levels = outputs[0]
        trends = outputs[1]
        seasons = outputs[2]


        levels_f = pm.math.concatenate([initial_level.reshape(1,1), levels[:-1]])
        trends_f = pm.math.concatenate([initial_trend.reshape(1,1), trends[:-1]])
        trends_f = trends_f * dampener ### pm.math.prod here?
        seasons_f = pm.math.concatenate([initial_szns, seasons[:-szn_len]])

        levels_and_trends = pm.math.stack([levels_f, trends_f])
        level_plus_trend = levels_and_trends.sum(axis=0)

        level_plus_trend_and_seasons = pm.math.stack([level_plus_trend, seasons_f])
        y_hats = level_plus_trend_and_seasons.prod(axis=0)

        sig = pm.HalfCauchy('sigma', beta=10)
#         y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[szn_len-1:-1, :]['y_obs_lag-1'])
        y_like = pm.Normal('y_like', mu=y_hats, sigma=sig, observed=zdf.iloc[:-1, :]['y_obs_lag-1'])
        
    map_estimate = pm.find_MAP(model=hw_model)
    
    return map_estimate, ys, is_covid_start